<a href="https://colab.research.google.com/github/SakuraPie/ai-model/blob/main/Sovits_f0%E4%B8%80%E9%94%AE%E5%90%88%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

**适用于本colab关联的sovits2.0模型，1.0/2.0模型、脚本均不通用**

**[1.0模型合成](https://colab.research.google.com/drive/14GT8uWM9IATjVylYNGGVXiKWHaDHZSTf)**

[一键制作数据集](https://colab.research.google.com/drive/1avWZ_N5BsQcq45XkwQkDpmp912CLZS0n?usp=drive_open#scrollTo=xx2oAf90btEy)

[一键训练](https://colab.research.google.com/drive/1DexYpwWIdD_RRqQ165l-YoWMzFAHIbPy?usp=drive_open)

**支持一键合成长时间的音频（5min以上），建议使用GPU（CPU比较慢）**

**请确保每一步均按顺序完成**

**本colab高度自动化，完全按提示操作即可合成，不要自己新建路径等；报错可能是网络原因，清除数据从头开始即可**

# 配置环境

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install --upgrade pip
!git clone -b colab https://github.com/IceKyrin/sovits_f0_infer
%cd sovits_f0_infer
!pip install -r requirements.txt
!mkdir pth
!mkdir raw
!mkdir results
%cd wav_temp
!mkdir input
!mkdir output
%cd ..

In [ ]:
!pip install demjson pydub pyworld
import logging
import os

import soundfile

from sovits import infer_tool
from sovits.infer_tool import Svc
from wav_temp import merge

# 加载模型

## 下载内容编码器

In [ ]:
# 这个东西是https://github.com/bshall/hubert/releases/tag/v0.1 的hubert-soft-0d54a1f4.pt，可以自己替换来源、但是不能换其他模型（路径自己改）。
#!gdown --id '1cA37nsiSnsouF2TJkaXb3_VoA-rbifTu' --output /content/sovits_f0_infer/pth/hubert.pt
!wget https://huggingface.co/spaces/xiaolang/sovits_midi_dev/resolve/main/hubert.pt -O /content/sovits_f0_infer/pth/hubert.pt

## 加载生成器

如果要**替换自己的模型**，请自行上传**配套的2.0版本的configs、G_xxx.pth**

**使用训练专栏输出到云盘的，改一下名称、无需更改路径**

将**自己的配置json（上一篇生成了的）**上传至/content/sovits_f0_infer/configs/文件夹
将**自己的模型（上一篇生成了的）**上传至/content/sovits_f0_infer/pth文件夹

In [ ]:
from google.colab import drive

#@markdown 是否使用谷歌盘内模型（不勾选则自动下载猫雷模型）
g_drive = False #@param {type:"boolean"}
if g_drive:
  drive.mount('/content/drive/')
  #@markdown 若使用，自行更改文件夹名、模型名
  config_path = "/content/drive/MyDrive/paimeng/config.json" #@param {type:"string"}
  model_path = '/content/drive/MyDrive/paimeng/G_xxx.pth' #@param {type:"string"}
else:
  # 这个东西是https://huggingface.co/spaces/innnky/soft-vits-singingvc 的G.pth（猫雷），可以换成自己的模型（必须是按照sovits方式训练出的其他角色模型）
  !wget https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/152_epochs.pth -O /content/sovits_f0_infer/pth/G.pth  
  config_path = "./configs/nyarumul.json"
  model_path = "/content/sovits_f0_infer/pth/G.pth"

# 加载sovits模型、参数
svc_model = Svc(model_path, config_path)

# 声音转换

支持{1、2}**任选一个方式**的声音转换！
支持**10s以上5分钟以内**的音频（再久合成时间会过长）。
上传到/content/sovits_f0_infer/raw文件夹，支持自动合成歌曲

使用[spleeter](https://github.com/deezer/spleeter)的2stems模式分离歌曲，自动生成这两个文件。（请自行阅读官方使用文档）

spleeter separate -p spleeter:2stems -o output audio_example.mp3

**结果自动输出至results文件夹。**自行下载，无预览
mp3为自动合成的带伴奏歌曲，out_vits为纯人声。

1、使用参考音频

In [ ]:
!gdown --id '10JQMPdzp0gjg9cVVersxVZWhIr4UwrFF' --output /content/sovits_f0_infer/raw/vocals.wav

2、使用上传音频

自行上传至raw文件夹（单声道，22050hz，wav格式）

In [ ]:
#@title 加载Google云端硬盘，挂云盘直接往raw拖文件也可以
from google.colab import drive
drive.mount('/content/drive')

3、合成音频

In [ ]:
import torchaudio
#@markdown 角色id——猫雷模型：0号为猫雷，2号为云灏，3号为即霜，4号为奕兰秋

#@markdown 源音频为女声，**目标音色为男声**时，**降3-6key**效果较好，反之同理

#@markdown 角色id
speaker_id = "4" #@param {type:"string"}
#@markdown **可为正负（升降n个半音）**
vc_transform = "-3" #@param {type:"string"}
#@markdown 人声文件名（不带.wav）
clean_name = "vocals" #@param {type:"string"}

out_audio_name = clean_name
# 可填写音源文件列表，音源文件格式为wav，放置于raw文件夹下
clean_names = [clean_name]
# 合成多少歌曲时，若半音数量不足、自动补齐相同数量（按第一首歌的半音）
trans = [int(vc_transform)]  # 加减半音数（可为正负）s
# 每首歌同时输出的speaker_id
id_list = [int(speaker_id)]



cut_time=30
infer_tool.fill_a_to_b(trans, clean_names)  # 自动补齐
input_wav_path = "./wav_temp/input"
out_wav_path = "./wav_temp/output"
infer_tool.mkdir([input_wav_path, out_wav_path])
print("mis连续超过10%时，考虑升降半音\n")
# 遍历列表
for clean_name, tran in zip(clean_names, trans):
    raw_audio_path = f"./raw/{clean_name}.wav"
    svc_model.format_wav(raw_audio_path)
    for spk_id in id_list:
        # 清除缓存文件
        infer_tool.del_temp_wav("./wav_temp")
        var_list = []
        mis_list = []
        out_audio_name = clean_name

        infer_tool.cut_wav(raw_audio_path, out_audio_name, input_wav_path, cut_time)
        count = 0
        file_list = os.listdir(input_wav_path)
        for file_name in file_list:
            raw_path = f"{input_wav_path}/{file_name}"
            out_path = f"{out_wav_path}/{file_name}"

            out_audio, out_sr = svc_model.infer(spk_id, tran, raw_path)
            soundfile.write(out_path, out_audio.cpu().numpy(), svc_model.target_sample)

            mistake, var = svc_model.calc_error(raw_path, out_path, tran)
            mis_list.append(mistake)
            var_list.append(var)
            count += 1
            print(f"{file_name}: {round(100 * count / len(file_list), 2)}%  mis:{mistake} var:{var}")
        print(
            f"分段误差参考：0.3优秀，0.5左右合理，少量0.8-1可以接受\n若偏差过大，请调整升降半音数；多次调整均过大、说明超出歌手音域\n半音偏差：{mis_list}\n半音方差：{var_list}")
        merge.run(out_audio_name)
        # 清除缓存文件
        infer_tool.del_temp_wav("./wav_temp")

In [ ]:
import IPython.display as ipd
import torchaudio
#@markdown 预览干声（自行进results查看文件名，带.wav，仅支持wav）
source_path = "/content/sovits_f0_infer/results/vocals.wav"  #@param {type:"string"}
audio,sr = torchaudio.load(source_path)
ipd.display(ipd.Audio(audio, rate=sr))